In [ ]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import linregress
from scipy.stats import wilcoxon


In [ ]:
def polynomial_regression(X, y, order=1, confidence=95, num=100):
    confidence = 1 - ((1 - (confidence / 100)) / 2)
    y_model = np.polyval(np.polyfit(X, y, order), X)
    residual = y - y_model
    n = X.size                     
    m = 2                          
    dof = n - m  
    t = stats.t.ppf(confidence, dof) 
    std_error = (np.sum(residual**2) / dof)**.5
    X_line = np.linspace(np.min(X), np.max(X), num)
    y_line = np.polyval(np.polyfit(X, y, order), X_line)
    ci = t * std_error * (1/n + (X_line - np.mean(X))**2 / np.sum((X - np.mean(X))**2))**.5
    return X_line, y_line, ci
def format_func_M(value, tick_number):
    # find number of multiples of pi/2
    if value % 10000000 == 0:
        value = '{:.0f}'.format(value / 1000000)
        return str(value) + "M"
    
def format_func_K(value, tick_number):
    # find number of multiples of pi/2
    if value % 1000 == 0:
        value = '{:.0f}'.format(value / 1000)
        return str(value) + "K"
    


In [ ]:
def fitDepthCounts(df, types, order): #types genes or junctions
    df.columns = ['level', 'count', 'depth', 'depth_unique', 'sample']
    X_line, y_line, ci = polynomial_regression(df['depth_unique'], df['count'], order=order)
    #sns.set(font_scale=1.5)
    #print(df.head())
    
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.set_facecolor("white")
    ax.patch.set_edgecolor('black')
    ax.patch.set_linewidth(2) 
    ax.grid(False)
    
    ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func_M))
    ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func_K))
    
    ax.scatter(df['depth_unique'], df['count'])
    ax.plot(X_line, y_line)
    plt.ylabel('# of ' + types)
    ax.fill_between(X_line, y_line - ci, y_line + ci, alpha=.5)
    
    #plt.grid(color='black', linestyle='-', linewidth=1)
    coef_array = np.polyfit(df['depth_unique'], df['count'], order)
    plt.show()
    return(coef_array)

Hypothalamus and GTEx

In [ ]:
typ = 'genes'

In [ ]:
path = ''#path to Figure 4 files

In [ ]:
#From 100 subsamples (green)
est_df = pd.read_csv(path + '/hypo_' + typ, sep = ' ', header = None, index_col = 0)
est_df.columns = [typ, 'depth', 'depth_unique']

#From the real data (blue - 1, orange - 2)
real_df = pd.read_csv(path + '/gtex_' + typ, sep = ' ', header = None, index_col = 0)
real_df.columns = [typ, 'depth', 'depth_unique']

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func_M))
ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func_K))
ax.set_facecolor("white")
ax.patch.set_edgecolor('black')
ax.patch.set_linewidth(2) 

#the number of genes from the real data
real_df.plot.scatter(x='depth_unique', y=typ, ax=ax, c='grey')


#Real data: degree of freedom = 1 (blue)
X_line_real_1, y_line_real_1, ci_real_1 = polynomial_regression(real_df['depth_unique'], real_df[typ], 1)
coef_array_real_1 = np.polyfit(real_df['depth_unique'], real_df[typ], 1)
p = np.poly1d([coef_array_real_1[0], coef_array_real_1[1]])
a_real_1 = p(10000000)
b_real_1 = p(25000000)
c_real_1 = p(50000000)
d_real_1 = p(100000000)
e_real_1 = p(150000000)
f_real_1 = p(200000000)
g_real_1 = p(250000000)

ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],
        [a_real_1,b_real_1,c_real_1,d_real_1,e_real_1,f_real_1, g_real_1], 
         c='#1E88E5', linestyle='--', linewidth=5)
ax.fill_between(X_line_real_1, y_line_real_1 - ci_real_1, y_line_real_1 + ci_real_1, alpha=.5)

#Real data: degree of freedom = 1 (orange)
X_line_real_2, y_line_real_2, ci_real_2 = polynomial_regression(real_df['depth_unique'], real_df[typ], 2)
coef_array_real_2 = np.polyfit(real_df['depth_unique'], real_df[typ], 2)
p = np.poly1d([coef_array_real_2[0], coef_array_real_2[1], coef_array_real_2[2]])
a_real_2 = p(10000000)
b_real_2 = p(25000000)
c_real_2 = p(50000000)
d_real_2 = p(100000000)
e_real_2 = p(150000000)
f_real_2 = p(200000000)
g_real_2 = p(200000000)

ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],
        [a_real_2,b_real_2,c_real_2,d_real_2,e_real_2,f_real_2, g_real_2], 
         c='orange', linestyle='--', linewidth=5)
ax.fill_between(X_line_real_2, y_line_real_2 - ci_real_2, y_line_real_2 + ci_real_2, alpha=.5)


#From 100 subsamples (green)
coef_array_est = np.polyfit(est_df['depth'], est_df[typ], 2)
p = np.poly1d([coef_array_est[0], coef_array_est[1], coef_array_est[2]])
a = p(10000000)
b = p(25000000)
c = p(50000000)
d = p(100000000)
e = p(150000000)
f = p(200000000)
f = p(200000000)
ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],[a,b,c,d,e,f, g], 
         c='#004D40', linestyle='--', linewidth=5)


plt.ylabel('# of ' + typ, size=30)
plt.xlabel('')
plt.xticks(size=25)
plt.yticks(size=25)
plt.ylim(0,)
plt.xlim(0,250000000)


#the baseline is a number of genes with AS with meadian seq depth
mean_seq_depth = real_df['depth_unique'].mean()
number_of_as = coef_array_est[0] * mean_seq_depth * mean_seq_depth + coef_array_est[1] * mean_seq_depth + coef_array_est[2]

plt.title ('')
plt.hlines(number_of_as, 0, 250000000, colors = 'black', linestyles='-')
plt.vlines(200000000, 0, 30000, linestyles='--', colors='black')
plt.show()
plt.show()

In [ ]:
print(number_of_as) #baseline
print(f) #green
print(f_real_1) #blue
print(f_real_2)#orange

In [ ]:
print(f_real_1 - number_of_as)
print(f_real_2 - number_of_as)
print(f - number_of_as)

In [ ]:
print((f_real_1 - number_of_as)/number_of_as*100)
print((f_real_2 - number_of_as)/number_of_as*100)
print((f - number_of_as)/number_of_as*100)


In [ ]:
typ = 'junctions' #genes or junctions
path = ''#path to Figure 4 genes

In [ ]:
#From 100 subsamples (green)
est_df = pd.read_csv(path + '/hypo_' + typ, sep = ' ', header = None, index_col = 0)
est_df.columns = [typ, 'depth', 'depth_unique']

#From the real data (blue - 1, orange - 2)
real_df = pd.read_csv(path + '/gtex_' + typ, sep = ' ', header = None, index_col = 0)
real_df.columns = [typ, 'depth', 'depth_unique']

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func_M))
ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func_K))
ax.set_facecolor("white")
ax.patch.set_edgecolor('black')
ax.patch.set_linewidth(2) 

#the number of genes from the real data
real_df.plot.scatter(x='depth_unique', y=typ, ax=ax, c='grey')


#Real data: degree of freedom = 1 (blue)
X_line_real_1, y_line_real_1, ci_real_1 = polynomial_regression(real_df['depth_unique'], real_df[typ], 1)
coef_array_real_1 = np.polyfit(real_df['depth_unique'], real_df[typ], 1)
p = np.poly1d([coef_array_real_1[0], coef_array_real_1[1]])
a_real_1 = p(10000000)
b_real_1 = p(25000000)
c_real_1 = p(50000000)
d_real_1 = p(100000000)
e_real_1 = p(150000000)
f_real_1 = p(200000000)
g_real_1 = p(250000000)

ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],
        [a_real_1,b_real_1,c_real_1,d_real_1,e_real_1,f_real_1, g_real_1], 
         c='#1E88E5', linestyle='--', linewidth=5)
ax.fill_between(X_line_real_1, y_line_real_1 - ci_real_1, y_line_real_1 + ci_real_1, alpha=.5)

#Real data: degree of freedom = 1 (orange)
X_line_real_2, y_line_real_2, ci_real_2 = polynomial_regression(real_df['depth_unique'], real_df[typ], 2)
coef_array_real_2 = np.polyfit(real_df['depth_unique'], real_df[typ], 2)
p = np.poly1d([coef_array_real_2[0], coef_array_real_2[1], coef_array_real_2[2]])
a_real_2 = p(10000000)
b_real_2 = p(25000000)
c_real_2 = p(50000000)
d_real_2 = p(100000000)
e_real_2 = p(150000000)
f_real_2 = p(200000000)
g_real_2 = p(200000000)

ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],
        [a_real_2,b_real_2,c_real_2,d_real_2,e_real_2,f_real_2, g_real_2], 
         c='orange', linestyle='--', linewidth=5)
ax.fill_between(X_line_real_2, y_line_real_2 - ci_real_2, y_line_real_2 + ci_real_2, alpha=.5)


#From 100 subsamples (green)
coef_array_est = np.polyfit(est_df['depth'], est_df[typ], 2)
p = np.poly1d([coef_array_est[0], coef_array_est[1], coef_array_est[2]])
a = p(10000000)
b = p(25000000)
c = p(50000000)
d = p(100000000)
e = p(150000000)
f = p(200000000)
g = p(200000000)
ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],[a,b,c,d,e,f, g], 
         c='#004D40', linestyle='--', linewidth=5)


plt.ylabel('# of ' + typ, size=30)
plt.xlabel('')
plt.xticks(size=25)
plt.yticks(size=25)
plt.ylim(0,)
plt.xlim(0,250000000)


#the baseline is a number of genes with AS with meadian seq depth
mean_seq_depth = real_df['depth_unique'].mean()
number_of_as = coef_array_est[0] * mean_seq_depth * mean_seq_depth + coef_array_est[1] * mean_seq_depth + coef_array_est[2]

plt.title ('')
plt.hlines(number_of_as, 0, 250000000, colors = 'black', linestyles='-')
plt.vlines(200000000, 0, 3000000, linestyles='--', colors='black')
plt.show()
plt.show()

In [ ]:
print(number_of_as) #baseline
print(f) #green
print(f_real_1) #blue
print(f_real_2)#orange

In [ ]:
print(f_real_1 - number_of_as)
print(f_real_2 - number_of_as)
print(f - number_of_as)

In [ ]:
print((f_real_1 - number_of_as)/number_of_as*100)
print((f_real_2 - number_of_as)/number_of_as*100)
print((f - number_of_as)/number_of_as*100)


TCGA

In [ ]:
typ = 'genes'
path = ''#path to Figure 4 files

In [ ]:
#From 100 subsamples (green)
est_df = pd.read_csv(path + '/tcga_est_' + typ, sep = ' ', header = None, index_col = 0)
est_df.columns = [typ, 'depth', 'depth_unique']

#From the real data (blue - 1, orange - 2)
real_df = pd.read_csv(path + '/tcga_' + typ, sep = ' ', header = None, index_col = 0)
real_df.columns = [typ, 'depth', 'depth_unique']

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func_M))
ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func_K))
ax.set_facecolor("white")
ax.patch.set_edgecolor('black')
ax.patch.set_linewidth(2) 

#the number of genes from the real data
real_df.plot.scatter(x='depth_unique', y=typ, ax=ax, c='grey')

#1-order estimation
X_line_real_1, y_line_real_1, ci_real_1 = polynomial_regression(real_df['depth_unique'], real_df[typ], 1)
coef_array_real_1 = np.polyfit(real_df['depth_unique'], real_df[typ], 1)
p = np.poly1d([coef_array_real_1[0], coef_array_real_1[1]])
a_real_1 = p(10000000)
b_real_1 = p(25000000)
c_real_1 = p(50000000)
d_real_1 = p(100000000)
e_real_1 = p(150000000)
f_real_1 = p(200000000)
g_real_1 = p(250000000)

ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],
        [a_real_1,b_real_1,c_real_1,d_real_1,e_real_1,f_real_1,g_real_1], 
         c='#1E88E5', linestyle='--', linewidth=5)
ax.fill_between(X_line_real_1, y_line_real_1 - ci_real_1, y_line_real_1 + ci_real_1, alpha=.5)

#2-order estimation
X_line_real_2, y_line_real_2, ci_real_2 = polynomial_regression(real_df['depth_unique'], real_df[typ], 2)
coef_array_real_2 = np.polyfit(real_df['depth_unique'], real_df[typ], 2)
p = np.poly1d(np.array([coef_array_real_2[0], coef_array_real_2[1], coef_array_real_2[2]], dtype=object))
ax.plot([10000000, 25000000, 50000000, 75000000, 100000000, 
         125000000, 150000000, 175000000, 200000000, 250000000],
        (p(10000000), p(25000000), p(50000000), p(75000000),
         p(100000000), p(125000000), p(150000000), p(175000000), p(200000000), p(250000000)), 
         c='orange', linestyle='--', linewidth=5, alpha = 0.3)
ax.fill_between(X_line_real_2, y_line_real_2 - ci_real_2, y_line_real_2 + ci_real_2, alpha=.3)



#the estimated line from the downsampled data
coef_array_est = np.polyfit(est_df['depth'], est_df[typ], 2)
p = np.poly1d([coef_array_est[0], coef_array_est[1], coef_array_est[2]])
a = p(10000000)
b = p(25000000)
c = p(50000000)
d = p(100000000)
e = p(150000000)
f = p(200000000)
g = p(250000000)

ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],
        [a,b,c,d,e,f,g], 
         c='#004D40', linestyle='--', linewidth=5)


plt.ylabel('# of ' + typ, size=30)
plt.xlabel('')
plt.xticks(size=25)
plt.yticks(size=25)
plt.ylim(0,)
plt.xlim(0,250000000)


#the baseline is a number of genes with AS with meadian seq depth
mean_seq_depth = real_df['depth_unique'].median()
number_of_as = coef_array_est[0] * mean_seq_depth * mean_seq_depth + coef_array_est[1] * mean_seq_depth + coef_array_est[2]

plt.title ('')
plt.hlines(number_of_as, 0, 250000000, colors = 'black', linestyles='-')
plt.vlines(200000000, 0, 30000, linestyles='--', colors='black')
plt.show()
plt.show()

In [ ]:
print(number_of_as) #baseline
print(f) #green
print(f_real_1) #blue


In [ ]:
print(f_real_1 - number_of_as)
print(f - number_of_as)

In [ ]:
print((f_real_1 - number_of_as)/number_of_as*100)
print((f - number_of_as)/number_of_as*100)

In [ ]:
typ = 'junctions' #genes or junctions

In [ ]:
#From 100 subsamples (green)
est_df = pd.read_csv(path + '/tcga_est_' + typ, sep = ' ', header = None, index_col = 0)
est_df.columns = [typ, 'depth', 'depth_unique']

#From the real data (blue - 1, orange - 2)
real_df = pd.read_csv(path + '/tcga_' + typ, sep = ' ', header = None, index_col = 0)
real_df.columns = [typ, 'depth', 'depth_unique']

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func_M))
ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func_K))
ax.set_facecolor("white")
ax.patch.set_edgecolor('black')
ax.patch.set_linewidth(2) 

#the number of genes from the real data
real_df.plot.scatter(x='depth_unique', y=typ, ax=ax, c='grey')

#1-order estimation
X_line_real_1, y_line_real_1, ci_real_1 = polynomial_regression(real_df['depth_unique'], real_df[typ], 1)
coef_array_real_1 = np.polyfit(real_df['depth_unique'], real_df[typ], 1)
p = np.poly1d([coef_array_real_1[0], coef_array_real_1[1]])
a_real_1 = p(10000000)
b_real_1 = p(25000000)
c_real_1 = p(50000000)
d_real_1 = p(100000000)
e_real_1 = p(150000000)
f_real_1 = p(200000000)
g_real_1 = p(250000000)

ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],
        [a_real_1,b_real_1,c_real_1,d_real_1,e_real_1,f_real_1,g_real_1], 
         c='#1E88E5', linestyle='--', linewidth=5)
ax.fill_between(X_line_real_1, y_line_real_1 - ci_real_1, y_line_real_1 + ci_real_1, alpha=.5)

#2-order estimation
X_line_real_2, y_line_real_2, ci_real_2 = polynomial_regression(real_df['depth_unique'], real_df[typ], 2)
coef_array_real_2 = np.polyfit(real_df['depth_unique'], real_df[typ], 2)
p = np.poly1d(np.array([coef_array_real_2[0], coef_array_real_2[1], coef_array_real_2[2]], dtype=object))
ax.plot([10000000, 25000000, 50000000, 75000000, 100000000, 
         125000000, 150000000, 175000000, 200000000, 250000000],
        (p(10000000), p(25000000), p(50000000), p(75000000),
         p(100000000), p(125000000), p(150000000), p(175000000), p(200000000), p(250000000)), 
         c='orange', linestyle='--', linewidth=5, alpha = 0.3)
ax.fill_between(X_line_real_2, y_line_real_2 - ci_real_2, y_line_real_2 + ci_real_2, alpha=.3)



#the estimated line from the downsampled data
coef_array_est = np.polyfit(est_df['depth'], est_df[typ], 2)
p = np.poly1d([coef_array_est[0], coef_array_est[1], coef_array_est[2]])
a = p(10000000)
b = p(25000000)
c = p(50000000)
d = p(100000000)
e = p(150000000)
f = p(200000000)
g = p(250000000)

ax.plot([10000000, 25000000, 50000000, 100000000, 150000000, 200000000, 250000000],
        [a,b,c,d,e,f,g], 
         c='#004D40', linestyle='--', linewidth=5)


plt.ylabel('# of ' + typ, size=30)
plt.xlabel('')
plt.xticks(size=25)
plt.yticks(size=25)
plt.ylim(0,)
plt.xlim(0,250000000)


#the baseline is a number of genes with AS with meadian seq depth
mean_seq_depth = real_df['depth_unique'].median()
number_of_as = coef_array_est[0] * mean_seq_depth * mean_seq_depth + coef_array_est[1] * mean_seq_depth + coef_array_est[2]

plt.title ('')
plt.hlines(number_of_as, 0, 250000000, colors = 'black', linestyles='-')
plt.vlines(200000000, 0, 3000000, linestyles='--', colors='black')
plt.show()
plt.show()

In [ ]:
print(number_of_as) #baseline
print(f) #green
print(f_real_1) #blue


In [ ]:
print(f_real_1 - number_of_as)
print(f - number_of_as)

In [ ]:
print((f_real_1 - number_of_as)/number_of_as*100)
print((f - number_of_as)/number_of_as*100)